In [28]:
# !pip install sklearn

In [67]:
import spacy
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [21]:
df = pd.read_csv('spam.csv')

In [32]:
df.shape

(5572, 3)

In [23]:
df.Category.value_counts()

ham     4825
spam     747
Name: Category, dtype: int64

In [24]:
df['spam'] = df['Category'].apply(lambda x: 1 if x == 'spam' else 0)

In [25]:
df.head()

,Category,Message,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [30]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.Message, df.spam, test_size=0.2)

In [34]:
X_train.shape, X_test.shape

((4457,), (1115,))

In [42]:
X_train[:4]

2545    She is our sister.. She belongs 2 our family.....
789     5 Free Top Polyphonic Tones call 087018728737,...
2709                              Nah, I'm a perpetual DD
4397    Dear :-/ why you mood off. I cant drive so i b...
Name: Message, dtype: object

In [43]:
y_train[:4]

2545    0
789     1
2709    0
4397    0
Name: spam, dtype: int64

In [53]:
from sklearn.feature_extraction.text import CountVectorizer

v = CountVectorizer()
X_train_cv = v.fit_transform(X_train.values)
X_train_cv

<4457x7695 sparse matrix of type '<class 'numpy.int64'>'
	with 59006 stored elements in Compressed Sparse Row format>

In [62]:
X_train_np = X_train_cv.toarray()
X_train_np[:4]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [63]:
v.get_feature_names_out().shape

(7695,)

In [64]:
v.get_feature_names_out()[1000:1050]

array(['apart', 'apartment', 'apes', 'apeshit', 'apnt', 'apologetic',
       'apologise', 'apologize', 'app', 'apparently', 'appeal', 'appear',
       'appendix', 'applausestore', 'applebees', 'apples', 'apply',
       'applyed', 'appointment', 'appointments', 'appreciate',
       'appreciated', 'appropriate', 'approve', 'approved', 'approx',
       'apps', 'appt', 'april', 'aproach', 'apt', 'aptitude', 'aquarius',
       'ar', 'arcade', 'archive', 'ard', 'are', 'area', 'aren', 'arent',
       'arestaurant', 'aretaking', 'areyouunique', 'argentina', 'argh',
       'argue', 'arguing', 'argument', 'arguments'], dtype=object)

In [65]:
v.vocabulary_

{'she': 6038,
 'is': 3725,
 'our': 4967,
 'sister': 6148,
 'belongs': 1302,
 'family': 2745,
 'hope': 3474,
 'of': 4855,
 'tomorrow': 6912,
 'pray': 5322,
 'her': 3394,
 'who': 7442,
 'was': 7333,
 'fated': 2763,
 'shoranur': 6074,
 'train': 6958,
 'incident': 3630,
 'lets': 4052,
 'hold': 3450,
 'hands': 3297,
 'together': 6896,
 'amp': 942,
 'fuelled': 3008,
 'by': 1576,
 'love': 4186,
 'concern': 1940,
 'prior': 5373,
 'grief': 3221,
 'pain': 5017,
 'pls': 5214,
 'join': 3811,
 'in': 3625,
 'dis': 2318,
 'chain': 1702,
 'pass': 5059,
 'it': 3737,
 'stop': 6448,
 'violence': 7245,
 'against': 853,
 'women': 7522,
 'free': 2955,
 'top': 6932,
 'polyphonic': 5254,
 'tones': 6915,
 'call': 1591,
 '087018728737': 81,
 'national': 4669,
 'rate': 5530,
 'get': 3091,
 'toppoly': 6934,
 'tune': 7028,
 'sent': 5982,
 'every': 2645,
 'week': 7384,
 'just': 3847,
 'text': 6737,
 'subpoly': 6509,
 'to': 6886,
 '81618': 644,
 'per': 5101,
 'pole': 5247,
 'unsub': 7131,
 '08718727870': 142,
 'nah'

In [66]:
v.get_feature_names_out()[6038]

'she'

In [69]:
# what are the indexes where value is not zero in 0th email

np.where(X_train_np[0] != 0) 

(array([ 853,  942, 1302, 1576, 1702, 1940, 2318, 2745, 2763, 3008, 3221,
        3297, 3394, 3450, 3474, 3625, 3630, 3725, 3737, 3811, 4052, 4186,
        4855, 4967, 5017, 5059, 5214, 5322, 5373, 6038, 6074, 6148, 6448,
        6896, 6912, 6958, 7245, 7333, 7442, 7522]),)

In [71]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(X_train_cv, y_train)

In [73]:
from sklearn.metrics import classification_report

X_test_cv = v.transform(X_test)
y_preds = model.predict(X_test_cv)

print(classification_report(y_test, y_preds))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       961
           1       0.96      0.91      0.93       154

    accuracy                           0.98      1115
   macro avg       0.97      0.95      0.96      1115
weighted avg       0.98      0.98      0.98      1115



In [74]:
# Building a NLP pipeline
from sklearn.pipeline import Pipeline

clf = Pipeline([('Vectorizer', CountVectorizer()), \
                ('Naive_Bayes', MultinomialNB())])

# No need to first convert data to numbers ,
# because the pipeline has countvectorizer and it can handle all conversions
clf.fit(X_train, y_train)

Pipeline(steps=[('Vectorizer', CountVectorizer()),
                ('Naive_Bayes', MultinomialNB())])

In [75]:
y_preds = clf.predict(X_test)

print(classification_report(y_test, y_preds))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       961
           1       0.96      0.91      0.93       154

    accuracy                           0.98      1115
   macro avg       0.97      0.95      0.96      1115
weighted avg       0.98      0.98      0.98      1115

